In [17]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [18]:
#Import the data:
heart_df = pd.read_csv('heart.csv')

In [19]:
#Analyse the data:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [20]:
#Analyse the data:
heart_df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [21]:
#Eliminate the outliers:
iso_forest = IsolationForest(random_state=0, contamination='auto').fit(heart_df)
iso_forest = iso_forest.fit(heart_df)
isof_outliers = iso_forest.predict(heart_df)
isoF_outliers_values = heart_df[iso_forest.predict(heart_df) == -1]
heart_df = heart_df.drop(isoF_outliers_values.index.values.tolist())
heart_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1


In [22]:
#Choose some classification algorithms:

X = heart_df.drop('target', axis = 1)
y = heart_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [23]:
lgr = LogisticRegression(solver = 'liblinear').fit(X_train, y_train)
knn = KNeighborsClassifier().fit(X_train,y_train)
dtc = DecisionTreeClassifier(random_state=0).fit(X_train,y_train)
rfc = RandomForestClassifier(random_state=0, verbose=False).fit(X_train, y_train)

models = [lgr, knn, dtc, rfc]
for model in models:
    score = -1 * cross_val_score(model, X, y, cv = 10, scoring ='neg_mean_absolute_error')
    print(f"MAE Score for {model}:\n", score.mean())   

MAE Score for LogisticRegression(solver='liblinear'):
 0.149802371541502
MAE Score for KNeighborsClassifier():
 0.35158102766798416
MAE Score for DecisionTreeClassifier(random_state=0):
 0.21916996047430826
MAE Score for RandomForestClassifier(random_state=0, verbose=False):
 0.14110671936758895


In [26]:
#Train the model:

lr = LogisticRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)
score = -1* cross_val_score(lr, X, y, cv = 8, scoring='neg_mean_absolute_error')

print("Mean Absolute error is:", score.mean())
print("The accuracy score of Logistric Regression Classifier is:", acc_score)

print("Classification Report:\n",classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

c:\users\timbur dinu\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\timbur dinu\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

Mean Absolute error is: 0.14547413793103448
The accuracy score of Logistric Regression Classifier is: 0.8478260869565217
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.83      0.85        24
           1       0.83      0.86      0.84        22

    accuracy                           0.85        46
   macro avg       0.85      0.85      0.85        46
weighted avg       0.85      0.85      0.85        46

Confusion Matrix:
 [[20  4]
 [ 3 19]]


c:\users\timbur dinu\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\timbur dinu\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

In [27]:
#Train the model:
rfc_pred = rfc.predict(X_test)
scores = -1 * cross_val_score(rfc, X, y, cv = 10, scoring ='neg_mean_absolute_error')
print("MAE is:", scores.mean())
print("Accuracy score is: ", accuracy_score(y_test, rfc_pred))
print("Classification Report:\n", classification_report(y_test, rfc_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rfc_pred))

MAE is: 0.14110671936758895
Accuracy score is:  0.8478260869565217
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.79      0.84        24
           1       0.80      0.91      0.85        22

    accuracy                           0.85        46
   macro avg       0.85      0.85      0.85        46
weighted avg       0.85      0.85      0.85        46

Confusion Matrix:
 [[19  5]
 [ 2 20]]


The accuracy of Logistic Regression model is about 85%.
Precision of 0.85/0.86, recall of 0.81/0.88 and f1_score of 0.83/0.87 shows that the model's performance is fairly good. 
Confusion matrix also shows results that supports the good performance of the above model.

On the flip side, RandomForestClassifier model has low MAE and fairly good accuracy of 88%. 
It has good scores for Precision, Recall and F1 Score as well. 
The confusion matrix values show good prediction results of the model too.
Therefore, we can observe that RandomForstClassifier is better performed when compared to LogisticRegression model. 
We can conclude that the RandomForestClassifier model seems to be the best model for our heart attack data prediction.